<a href="https://colab.research.google.com/github/sur-sakthy/data-visualisation-project/blob/main/data/steamcharts/data_gathering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Getting data from a web api 
The API I've used to fetch data from is called [Steamcharts](https://rapidapi.com/capusso/api/steamcharts) hosted on [RapidAPI Hub](https://rapidapi.com/hub).

The data provided by Steamcharts is used on their [website](https://steamcharts.com/) to display Trending, Top Games and Top Records. 

I intend to use the Avg. Players count over time to visualise the hype and interactivity around the game.

In [ ]:
# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth, files
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
import pandas as pd
import requests
import json
import time

I've extracted out the unique game ids (using tableau) from the [steam_reviews](https://www.kaggle.com/datasets/najzeko/steam-reviews-2021) dataset which contains around 21 million user reviews of around 300 different games on Steam.

In [ ]:
shareable_link = 'https://drive.google.com/file/d/1WiesJNCA-clZHwvKVKGMRU8QNWLtlTk1/view?usp=share_link'

downloaded = drive.CreateFile({'id':'1WiesJNCA-clZHwvKVKGMRU8QNWLtlTk1'}) 
downloaded.GetContentFile('unique_app_ids.csv')  
data = pd.read_csv('unique_app_ids.csv')
data

App Id
0    1291340
1    1289310
2    1240210
3    1229490
4    1225330
..       ...
310     2870
311      620
312      420
313      240
314       70

[315 rows x 1 columns]

## Fetching game stats
I am now using the game ids to fetch stats.

There is a quota on the number of requests that can be sent to the rapid api server - 1000 requests per month for the free version. 

---
> ⚠ **If you intend to run this code, please replace the api key with yours**

In [ ]:
steamcharts = 'https://steamcharts.p.rapidapi.com/api/v1/games/'
headers = {
    'X-RapidAPI-Key': '4937138c4amshd89d92d21921ad8p16a3acjsnf8fad18ba097',
    'X-RapidAPI-Host': 'steamcharts.p.rapidapi.com'
}

dataset = pd.DataFrame()
for index, row in data.iterrows():
  # print(row['App Id'])
  response = requests.get(steamcharts + str(row['App Id']), headers=headers)
  if response.status_code != 200:
    print("Error fetching for App Id %d : %d" %(row['App Id'], response.status_code))
  else:
    response_object = json.loads(response.content)
    response_data = pd.DataFrame(response_object['stats'])
    response_data['id'] = response_object['id']
    response_data['name'] = response_object['name']
    response_data['playingTwelveHoursAgo'] = response_object['playingTwelveHoursAgo']
    response_data['twentyFourHourPeek'] = response_object['twentyFourHourPeek']
    response_data['allTimePeek'] = response_object['allTimePeek']
    dataset = dataset.append(response_data)
  time.sleep(6)


## Viewing a sample of the obtained dataset

In [ ]:
dataset

month  averagePlayers    gain  gainInPercent  peakPlayers  \
0      Last 30 Days          118.16   38.60          48.50          437   
1    September 2022           79.56   -5.66          -6.64          384   
2       August 2022           85.22   -8.35          -8.93          169   
3         July 2022           93.58   -8.48          -8.31          208   
4         June 2022          102.05  -61.94         -37.77          213   
..              ...             ...     ...            ...          ...   
119   November 2012          157.47    2.12           1.37          356   
120    October 2012          155.35  -61.41         -28.33          345   
121  September 2012          216.76  -19.78          -8.36          464   
122     August 2012          236.53 -115.83         -32.87          419   
123       July 2012          352.36     NaN            NaN          674   

          id        name  playingTwelveHoursAgo  twentyFourHourPeek  \
0    1291340  Townscaper                     54                  93   
1    1291340  Townscaper                     54                  93   
2    1291340  Townscaper                     54                  93   
3    1291340  Townscaper                     54                  93   
4    1291340  Townscaper                     54                  93   
..       ...         ...                    ...                 ...   
119       70   Half-Life                    460                 590   
120       70   Half-Life                    460                 590   
121       70   Half-Life                    460                 590   
122       70   Half-Life                    460                 590   
123       70   Half-Life                    460                 590   

     allTimePeek  
0           1162  
1           1162  
2           1162  
3           1162  
4           1162  
..           ...  
119        10480  
120        10480  
121        10480  
122        10480  
123        10480  

[21287 rows x 10 columns]

## Exporting the dataset

In [ ]:
dataset.to_csv('steam_charts.csv', sep=',', encoding='utf-8')

## Downloading the dataset

In [ ]:
files.download('steam_charts.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>